In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import os
import numpy as np
import pandas as pd
import wandb
import plotly.express as px

# Pytorch implementation

In [2]:
use_wandb = False

In [3]:
config = {
    "learning_rate": 1e-3,
    "dataset": "Titanic",
    "epochs": 500,
    "batch_size": 16,
    "loss": "binary cross-entropy",
    "optimizer": "Adam"
}

In [4]:
if use_wandb:
    wandb.login()
    wandb.init(project="assignment-1", name="pytorch", reinit=True)
    wandb.config = config

In [5]:
# device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device = 'cpu'
print(f"Using {device} device")

Using cpu device


## Loading the dataset

In [6]:
class TitanicDataset(Dataset):
    def __init__(self, data_dir):
        self.labels = pd.read_csv(os.path.join(data_dir, 'labels.csv')).to_numpy(dtype='float32')
        self.data = pd.read_csv(os.path.join(data_dir, 'data.csv')).to_numpy(dtype='float32')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = torch.from_numpy(self.data[idx])
        label = torch.from_numpy(self.labels[idx])
        # data = self.data[idx]
        # label = self.labels[idx]

        return data, label

In [7]:
train_data = TitanicDataset(data_dir='data/train/')
val_data = TitanicDataset(data_dir='data/val/')
test_data = TitanicDataset(data_dir='data/test/')

In [8]:
features, label = train_data.__getitem__(idx=0)
print(features, label)

tensor([2.0000, 0.3322, 1.0000, 1.0000, 3.5115]) tensor([1.])


In [9]:
train_dataloader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=config['batch_size'], shuffle=True)

In [10]:
for batch, (x, y) in enumerate(train_dataloader):
    print(batch, x.shape, y.shape)
    break

0 torch.Size([16, 5]) torch.Size([16, 1])


## Defining the model

In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.linear(x)
        return out

In [12]:
model = NeuralNetwork().to(device)
print(model)

if use_wandb: 
    wandb.config['model'] = model.__dict__['_modules']['linear']

NeuralNetwork(
  (linear): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


## Training

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

In [14]:
# wandb.watch(model, loss_fn, log='all', log_freq=10)

In [15]:
def calculate_acc(y_pred, y) -> float:
    y_pred = y_pred>0.5

    # print(((y == y_pred).sum().item()))
    return ((y == y_pred).sum().item())/len(y)

In [16]:
loss_graph = []
acc_graph = []

def train(dataloader, model, loss_fn, optimizer, epoch):
    model.train()
    avg_loss, avg_acc = 0, 0
    curr = 0
    for x, y in dataloader:
        curr += len(y)
        x, y = x.to(device), y.to(device)

        # Get prediction and compute loss
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        avg_loss += loss.item()

        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_acc += calculate_acc(y_pred, y)
        # print(f'loss: {loss.item():>5f} acc: {100*correct:>5f} | {curr}/{len(dataloader.dataset)}')

    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)
    
    loss_graph.append(avg_loss)
    acc_graph.append(avg_acc)
    
    # if (epoch % 50 == 0):
    print(f"Epoch {epoch}")
    print(f'avg loss: {avg_loss:>5f} avg acc: {avg_acc:>5f}')
    
    if use_wandb:
        wandb.log({'loss': avg_loss})
        wandb.log({'acc': avg_acc})

In [17]:
val_loss_graph = []
val_acc_graph = []

def val(dataloader, model, loss_fn, epoch):    
    model.eval()
    avg_loss, avg_acc = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            avg_loss += loss_fn(y_pred, y).item()
            avg_acc += calculate_acc(y_pred, y)
    
    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)

    val_loss_graph.append(avg_loss)
    val_acc_graph.append(avg_acc)
    
    if (epoch % 50 == 0):
        print(f'avg val loss: {avg_loss:>5f} avg val acc: {avg_acc:>5f}')
        print('-------------------------------')
    
    if use_wandb:
        wandb.log({'val_loss': avg_loss})
        wandb.log({'val_acc': avg_acc})

In [18]:
epochs = config['epochs']
for epoch in range(epochs):
    # print(f"Epoch {epoch+1}")
    train(train_dataloader, model, loss_fn, optimizer, epoch)
    val(val_dataloader, model, loss_fn, epoch)
    # print('-------------------------------')

Epoch 0
avg loss: 0.000000 avg acc: 0.601674
avg val loss: 0.000000 avg val acc: 0.603350
-------------------------------
Epoch 1
avg loss: 0.000000 avg acc: 0.596688
Epoch 2
avg loss: 0.000000 avg acc: 0.599181
Epoch 3
avg loss: 0.000000 avg acc: 0.600427
Epoch 4
avg loss: 0.000000 avg acc: 0.600427
Epoch 5
avg loss: 0.000000 avg acc: 0.599181
Epoch 6
avg loss: 0.000000 avg acc: 0.601674
Epoch 7
avg loss: 0.000000 avg acc: 0.600427
Epoch 8
avg loss: 0.000000 avg acc: 0.600427
Epoch 9
avg loss: 0.000000 avg acc: 0.597934
Epoch 10
avg loss: 0.000000 avg acc: 0.601674
Epoch 11
avg loss: 0.000000 avg acc: 0.601674
Epoch 12
avg loss: 0.000000 avg acc: 0.599181
Epoch 13
avg loss: 0.000000 avg acc: 0.597934
Epoch 14
avg loss: 0.000000 avg acc: 0.596688
Epoch 15
avg loss: 0.000000 avg acc: 0.597934
Epoch 16
avg loss: 0.000000 avg acc: 0.599181
Epoch 17
avg loss: 0.000000 avg acc: 0.600427
Epoch 18
avg loss: 0.000000 avg acc: 0.596688
Epoch 19
avg loss: 0.000000 avg acc: 0.600427
Epoch 20
avg 

In [19]:
if use_wandb:
    wandb.finish()

In [20]:
fig = px.line({'loss': loss_graph, 'val_loss': val_loss_graph})
fig.show()

In [21]:
fig = px.line({'acc': acc_graph, 'val_acc': val_acc_graph})
fig.show()